In [117]:
include("utils/preprocessing.jl")
include("utils/model_evaluation.jl")
include("utils/data_loader.jl")


Main.DataLoader

In [118]:
using .DataLoader

data = DataLoader.load_data("dataset/star_classification.csv");
println(data[1,:])
println(size(data,2))


Any["obj_ID", "alpha", "delta", "u", "g", "r", "i", "z", "run_ID", "rerun_ID", "cam_col", "field_ID", "spec_obj_ID", "class", "redshift", "plate", "MJD", "fiber_ID"]
18


In [119]:
using .Preprocessing

inputs, targets = Preprocessing.preprocess_data(data)
println(inputs[1,:])
println(targets[1,:])

Float32[0.9989741, 0.9992305, 0.6196673, 0.4961512, 0.9990908, 0.54153734, 0.6, 0.118609406, 0.81344813, 0.129371, 0.81345165, 0.8990989, 0.7247247]
Bool[1, 0, 0]


In [120]:
print(count(targets .== [1 0 0]),'\n')
print(count(targets .== [0 1 0]),'\n')
print(count(targets .== [0 0 1]))


94805
94805
94805